In [39]:

import pandas as pd
import numpy as np
from collections import defaultdict
from surprise import Reader
from surprise import KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate
from sklearn.utils import shuffle

from surprise import SVD


## useful link for tutorial

https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea

https://surprise.readthedocs.io/en/stable/prediction_algorithms.html

##  for clarity's sake the number of reviewers who gave valid review to anime movie, was about 24 million viewers (or reviewers)
* I used excel in the beginning to do some calculations like this

* other possible ways to simple cut off useless data would be to disregard any movies from the analysis, which had reviewer count less than a certain number of reviewers, because it could be biased (trolling movie review, or niche based movie where reviews are less descriptive of the true situation)
* I didnt do anything about it though, at least yet
* I will attempt to do all processing inside this notebook
* im not totally sure if I remembered to do all that preprocessing in this notebook, I may or may have done something inside excel already, I forgot....

## problem is the size of the huge size rating.csv

### we need to limit the problem scope substantially

* select 500 most popularly reviewed anime movies only
* make those 500 movies the rows of the dataframe
* select randomly 300 reviewer persons for each of the  movies
* make those reviewer persons the columns of the dataframe

* select only popular movies reviewer persons > 1000
* excel was unable to open the rating.csv so we must limit ourselves in this regard, that rating dataframe must be reduced in size in a logical way...



In [40]:
path0 = r'C:\Users\Lauri\Documents\HealthTechnology Mathematics&Methods\Methods Lectures\anime(1).csv'
path1 = r'C:\Users\Lauri\Documents\HealthTechnology Mathematics&Methods\Methods Lectures\rating(1).csv'
movies_df = pd.read_csv(path0)
ratings_df = pd.read_csv(path1)




MOVIESAMOUNT = 300
CUTOFF = 1000 #amount of reviewers
SEED = 42
REVIEWERS = 200



movies_df = movies_df[movies_df.type=='Movie']
movies_df = movies_df[movies_df.members > CUTOFF]

#randomly sample the 500 movies from these most popular ones
movies_df = movies_df.sample(n=MOVIESAMOUNT, random_state = SEED)

#fill blank values for rating
movies_df["rating"] = movies_df["rating"].astype(float)
movies_df["rating"].fillna(movies_df["rating"].median(),inplace = True)



    
print(movies_df.describe())
#print(movies_df.type.unique())
#print(movies_df.members.unique())


           anime_id      rating        members
count    300.000000  300.000000     300.000000
mean    9749.756667    7.247633   23553.833333
std    10121.687481    0.722169   38947.991781
min       31.000000    2.670000    1001.000000
25%     2118.750000    6.922500    2371.250000
50%     5126.500000    7.330000    7577.000000
75%    13916.000000    7.640000   28219.000000
max    34161.000000    9.050000  339556.000000


In [41]:
movies_df.head()

,anime_id,name,genre,type,episodes,rating,members
3280,2746,Vexille: 2077 Nihon Sakoku,"Action, Military, Sci-Fi",Movie,1,7.04,14754
292,4107,Tengen Toppa Gurren Lagann Movie: Gurren-hen,"Action, Comedy, Mecha",Movie,1,8.22,72933
630,5310,Macross F Movie 1: Itsuwari no Utahime,"Action, Mecha, Military, Music, Romance, Sci-F...",Movie,1,7.93,20637
3324,2118,Captain Tsubasa: Europe Daikessen,"Action, Shounen, Sports",Movie,1,7.02,2819
2172,1901,11-nin Iru!,"Action, Adventure, Drama, Mystery, Romance, Sc...",Movie,1,7.33,4311


## drop useless columns for our anime movies analysis purposes
* drop  columns episodes, members, type

In [42]:
movies_df = movies_df.drop(['episodes', 'members', 'type'], axis=1)
movies_df.head()

,anime_id,name,genre,rating
3280,2746,Vexille: 2077 Nihon Sakoku,"Action, Military, Sci-Fi",7.04
292,4107,Tengen Toppa Gurren Lagann Movie: Gurren-hen,"Action, Comedy, Mecha",8.22
630,5310,Macross F Movie 1: Itsuwari no Utahime,"Action, Mecha, Military, Music, Romance, Sci-F...",7.93
3324,2118,Captain Tsubasa: Europe Daikessen,"Action, Shounen, Sports",7.02
2172,1901,11-nin Iru!,"Action, Adventure, Drama, Mystery, Romance, Sc...",7.33


## select ratings only ratings, from allowed anime_id movies

* it is done by using anime_id as a selection key of sorts as follows
* also drop ratings whose rating was negative (i.e. from kaggle notebooks I saw that negative rating means it was not really available the actual rating)

In [43]:
#we select reviewers by indexing into there by anime_id
ratings_df = ratings_df[ ratings_df.anime_id.isin(movies_df.anime_id) ]

#sort ratings by anime_id, probably not necessary step, but its nicer to look at 
ratings_df = ratings_df.sort_values('anime_id')

#drop the rating=-1 rows
ratings_df = ratings_df[ratings_df.rating >= 1.0]

ratings_df.head(25)


,user_id,anime_id,rating
4942571,47361,31,10
5494843,51772,31,6
2250726,21709,31,6
167923,1670,31,9
3931741,37104,31,8
2760288,25961,31,8
6320010,58748,31,10
5787956,54159,31,5
7473272,69821,31,8
5260266,49916,31,8


## prepare to further reduce ratings_df size
* we select for each unique anime_id, only certain j amount of movie reviews for each of those unique movies
* that way we are able to compute the algorithm without crashing

In [44]:
##for each anime_id, select only N rows and concatenate into dataframe
## that is the strategy for further culling the amount of data from the ratings_df
## so that our algorithmh doesnt crash our computer because ratings_df is too large
uniqueAnimeMovies = ratings_df.anime_id.unique()
uniqueAnimeMovies




array([   31,    47,    54,    88,   164,   187,   219,   371,   372,
         393,   415,   416,   418,   430,   462,   464,   465,   502,
         512,   515,   520,   572,   711,   723,   732,   814,   815,
         844,   867,   868,   882,   893,   895,   897,   899,   936,
        1006,  1010,  1029,  1049,  1096,  1118,  1119,  1140,  1141,
        1160,  1256,  1287,  1365,  1367,  1370,  1400,  1432,  1626,
        1681,  1723,  1760,  1773,  1777,  1793,  1815,  1861,  1901,
        1911,  1920,  1922,  1983,  1989,  2033,  2071,  2077,  2098,
        2107,  2111,  2118,  2119,  2132,  2141,  2176,  2199,  2202,
        2264,  2265,  2286,  2312,  2392,  2408,  2418,  2470,  2490,
        2491,  2493,  2498,  2499,  2621,  2648,  2664,  2665,  2666,
        2669,  2672,  2677,  2695,  2746,  2759,  2760,  2761,  2811,
        2840,  2847,  2848,  2890,  2922,  2948,  3016,  3038,  3067,
        3069,  3087,  3120,  3186,  3220,  3475,  3508,  3602,  3603,
        3782,  3783,

In [45]:
##attempt to resample only 5 reviews per each animeMovie into the new dataframe, they should be from random people
curRow=0
df_newsample=pd.DataFrame([],columns=ratings_df.columns)
for i in uniqueAnimeMovies:
    new_sample = ratings_df[ ratings_df[ 'anime_id' ] == i ].sample(n=5, replace=True)
    df_newsample=pd.concat( [df_newsample, new_sample], axis=0 )
    
    
print(df_newsample.head(30))    

        user_id anime_id rating
7043574   65809       31     10
5824710   54459       31      6
5435046   51316       31      8
5232346   49708       31      5
5034155   48187       31      8
4351332   41113       47     10
6490670   59977       47     10
6945587   64797       47     10
1767854   17115       47     10
1266425   11977       47      6
6296035   58601       54      7
5813771   54361       54      8
936895     8452       54      8
5538377   52086       54      7
5606101   52625       54      8
5527215   52014       88     10
2820298   26439       88      7
3503257   32417       88      4
2290443   22062       88      7
7121672   66429       88      8
6494897   60014      164      7
3007362   27902      164     10
4281528   40539      164     10
2041151   19754      164      9
332008     3389      164      8
2787851   26177      187      8
4035942   38275      187      8
6540520   60398      187      8
827146     7602      187      7
4023546   38148      187      8


In [46]:
df_newsample.describe()

,user_id,anime_id,rating
count,1430,1430,1430
unique,1196,286,10
top,42635,4095,8
freq,10,5,373


## now we have can finally reduce the ratings_df size into manageable size
* simple assignment into ratings_df from our temporary dataframe

In [47]:
ratings_df = df_newsample
ratings_df.head(30)



,user_id,anime_id,rating
7043574,65809,31,10
5824710,54459,31,6
5435046,51316,31,8
5232346,49708,31,5
5034155,48187,31,8
4351332,41113,47,10
6490670,59977,47,10
6945587,64797,47,10
1767854,17115,47,10
1266425,11977,47,6


In [48]:
ratings_df.describe()

,user_id,anime_id,rating
count,1430,1430,1430
unique,1196,286,10
top,42635,4095,8
freq,10,5,373


## here, we will start the join of the ratings_df and movies_df, into merged_df

In [49]:
# Construct reader
reader = Reader(rating_scale=(1, 10))

In [50]:
movies_df.reset_index(drop=True, inplace=True)
ratings_df.reset_index(drop=True, inplace=True) #reset the indeces just to prevent any silly problems in future

In [51]:
# Join the two dataframes on the anime_id columns

merged_df = ratings_df.merge(movies_df, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
merged_df.rename(columns = {'rating_user':'user_rating'}, inplace = True)
merged_df.rename(columns = {'rating':'avg_rating'}, inplace = True)
merged_df.rename(columns = {'name':'movie_name'}, inplace = True)
merged_df.head(10)

,user_id,anime_id,user_rating,movie_name,genre,avg_rating
0,65809,31,10,Neon Genesis Evangelion: Death &amp; Rebirth,"Drama, Mecha, Psychological, Sci-Fi",7.51
1,54459,31,6,Neon Genesis Evangelion: Death &amp; Rebirth,"Drama, Mecha, Psychological, Sci-Fi",7.51
2,51316,31,8,Neon Genesis Evangelion: Death &amp; Rebirth,"Drama, Mecha, Psychological, Sci-Fi",7.51
3,49708,31,5,Neon Genesis Evangelion: Death &amp; Rebirth,"Drama, Mecha, Psychological, Sci-Fi",7.51
4,48187,31,8,Neon Genesis Evangelion: Death &amp; Rebirth,"Drama, Mecha, Psychological, Sci-Fi",7.51
5,41113,47,10,Akira,"Action, Adventure, Horror, Military, Sci-Fi, S...",8.15
6,59977,47,10,Akira,"Action, Adventure, Horror, Military, Sci-Fi, S...",8.15
7,64797,47,10,Akira,"Action, Adventure, Horror, Military, Sci-Fi, S...",8.15
8,17115,47,10,Akira,"Action, Adventure, Horror, Military, Sci-Fi, S...",8.15
9,11977,47,6,Akira,"Action, Adventure, Horror, Military, Sci-Fi, S...",8.15


In [52]:
merged_df.describe()

,avg_rating
count,1430.000000
mean,7.255070
std,0.727872
min,2.670000
25%,6.900000
50%,7.330000
75%,7.660000
max,9.050000


## I think we can build the basic system for the KNN algorithm

#### then we can have a new user who wants to know what are some good animes, but hes an anime noob, so that he needs recommender system to help him

#### then, we can select a list of anime genres, and find a single user from the data with same genres

#### then, we get the top choices of that person already existing in the dataset

#### otherwise the recommender system seems a little bit useless because new users cannot query recommendations from the system ???!!!

* Knn user based algorithm can still be valuable algorithm for your existing database of subscribers or users, as long as you have data on all of those users...

* I would have loved to get that backhanded way of matching a new anime user into an existing anime watcher, and then recommending movies to that new anime user, but I ran out of time

* I will simply print out the movies for existing users in the users database and each movie recommendation for those people

* I believe that this type of printing will suffice for this user-based KNN algorithm

In [53]:
# This block copied from Surprise documentation at
# http://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


"""top_n = get_top_n(predictions, n=10)
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])"""


'top_n = get_top_n(predictions, n=10)\n# Print the recommended items for each user\nfor uid, user_ratings in top_n.items():\n    print(uid, [iid for (iid, _) in user_ratings])'

In [54]:
# Generate surprise Dataset
data = Dataset.load_from_df(merged_df[['user_id','anime_id','user_rating']], reader)
print(data)

In [55]:
trainset = data.build_full_trainset()
sim_options = {'name': 'cosine',
               'user_based': True  # compute  similarities between users
               }


algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [56]:
## simple hard coded check if some user exists, I think???
## not particularly significant piece of code, I think

## ALSO IT IS NOT CONSIDERED A BUG
## if that user is not in the database because this is the hardcoded uid, and hardcoded aid
## and our ratings_df changes because of randomized sampling of those reviews

uid=14045
aid=31
pred = algo.predict(uid, aid, verbose=True)

user: 14045      item: 31         r_ui = None   est = 7.46   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}


In [57]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

# Example:output
#print(predictions) 


"""NOTE if you print the predictions out, you will get notebook error for some reason
THE COMPUTATION WILL STILL BE EXECUTED in algo.test(), but the printing will crash it

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)


"""

'NOTE if you print the predictions out, you will get notebook error for some reason\nTHE COMPUTATION WILL STILL BE EXECUTED in algo.test(), but the printing will crash it\n\nIOPub data rate exceeded.\nThe notebook server will temporarily stop sending output\nto the client in order to avoid crashing it.\nTo change this limit, set the config variable\n`--NotebookApp.iopub_data_rate_limit`.\n\nCurrent values:\nNotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)\nNotebookApp.rate_limit_window=3.0 (secs)\n\n\n'

In [58]:
## helper function to print the movie names, based on the  movieIDs or IIDs in our case, I believe that this is the case
## so that uid == userID basically
## so that iid == animeID basically
## we basically execute "sql search" into the merged_df
## such that we search into it by using iid as animeID to select row, then we select that row's own movieName cell

def getMovieName(iid):
    mov_name= merged_df.loc[  merged_df[ 'anime_id' ]== iid, 'movie_name' ].iloc[0] 
    return mov_name
        ## note that the .iloc[0] is necessary to only return ONE search result, 
        ## instead of multiple of the same movie name

In [59]:
## get some frigging movies or something out from the recommender system!!! finally

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

65809 [47, 54, 88, 164, 187, 219, 371, 372, 393, 415]
54459 [47, 54, 88, 164, 187, 219, 371, 372, 393, 415]
51316 [47, 54, 88, 164, 187, 219, 371, 372, 393, 415]
49708 [2312, 47, 54, 88, 164, 187, 219, 371, 372, 393]
48187 [47, 54, 88, 164, 187, 219, 371, 372, 393, 415]
41113 [31, 54, 88, 164, 187, 219, 371, 372, 393, 415]
59977 [31, 54, 88, 164, 187, 219, 371, 372, 393, 415]
64797 [31, 54, 88, 164, 187, 219, 371, 372, 393, 415]
17115 [31, 54, 88, 164, 187, 219, 371, 372, 393, 415]
11977 [31, 54, 88, 164, 187, 219, 371, 372, 393, 415]
58601 [31, 47, 88, 164, 187, 219, 371, 372, 393, 415]
54361 [31, 47, 88, 164, 187, 219, 371, 372, 393, 415]
8452 [31, 47, 88, 164, 187, 219, 371, 372, 393, 415]
52086 [31, 47, 88, 164, 187, 219, 371, 372, 393, 415]
52625 [31, 47, 88, 164, 187, 219, 371, 372, 393, 415]
52014 [31, 47, 54, 164, 187, 219, 371, 372, 393, 415]
26439 [31, 47, 54, 164, 187, 219, 371, 372, 393, 415]
32417 [31, 47, 54, 164, 187, 219, 371, 372, 393, 415]
22062 [31, 47, 54, 164, 187,

23286 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
66133 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
33723 [5310, 31, 47, 54, 88, 164, 187, 219, 371, 372]
16139 [5310, 31, 47, 54, 88, 164, 187, 219, 371, 372]
43652 [5310, 31, 47, 54, 88, 164, 187, 219, 371, 372]
19098 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
61910 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
65840 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
28594 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
17635 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
43844 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
71792 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
19130 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
439 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
22985 [31, 47, 54, 88, 164, 187, 219, 371, 372, 393]
71490 [30711, 31, 47, 54, 88, 164, 187, 219, 371, 372]
58312 [30711, 31, 47, 54, 88, 164, 187, 219, 371, 372]
28490 [30711, 31, 47, 54, 88, 164, 187, 219, 371, 372]
41658 [30711, 31, 47, 54, 88, 164, 187,

In [60]:
print(getMovieName(31))

"""# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (getMovieName(iid), _) in user_ratings])"""


Neon Genesis Evangelion: Death &amp; Rebirth


'# Print the recommended items for each user\nfor uid, user_ratings in top_n.items():\n    print(uid, [iid for (getMovieName(iid), _) in user_ratings])'

In [61]:

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [getMovieName(iid) for (iid, _) in user_ratings])
    print('')

65809 ['Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne', 'Tonari no Yamada-kun']

54459 ['Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne', 'Tonari no Yamada-kun']

51316 ['Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne', 'Tonari no Yamada-kun']

49708 ['Daishizen no Majuu: Bagi', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcap

7602 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne', 'Tonari no Yamada-kun']

38148 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne', 'Tonari no Yamada-kun']

6360 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne', 'Tonari no Yamada-kun']

20527 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third St

26141 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

49393 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

57564 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

55360 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Monono

73135 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

55440 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

67239 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

57620 ['Wake Up, Girls! Beyond the Bottom', 'Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie

54258 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

44468 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

65732 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

4117 ['Rockman.EXE Movie: Hikari to Yami no Program', 'Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Apples

...

14676 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

30908 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

17801 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

16705 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Monono

26566 ['Anime Tenchou Movie', 'Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card']

45010 ['Anime Tenchou Movie', 'Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card']

44429 ['Anime Tenchou Movie', 'Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card']

27816 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobil

28521 ['Nerawareta Gakuen', 'Garo Movie: Divine Flame', 'Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1']

66055 ['Nerawareta Gakuen', 'Tantei Opera Milky Holmes Movie: Gyakushuu no Milky Holmes', 'Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1']

32058 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

63373 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91',

19643 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

58514 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

20824 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card', 'Escaflowne']

21329 ['Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Monono

66118 ['Harmony', 'Precure All Stars Movie New Stage: Mirai no Tomodachi', 'Takanashi Rikka Kai: Chuunibyou demo Koi ga Shitai! Movie', 'Harmonie', 'Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage']

21893 ['Major Movie: Yuujou no Winning Shot', 'Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card']

14805 ['Major Movie: Yuujou no Winning Shot', 'Neon Genesis Evangelion: Death &amp; Rebirth', 'Akira', 'Appleseed (Movie)', 'Mobile Suit Gundam F91', 'Mononoke Hime', 'Initial D Third Stage', 'Kidou Senkan Nadesico: The Prince of Darkness', 'Cardcaptor Sakura Movie 1', 'Cardcaptor Sakura Movie 2: Fuuin Sareta Card']

68315 ['Major Movie: Yuujou no Winning Shot', 'Neon Genesis Evangel

## results overall from exercise

* This was a difficult assignment in some sense, because preprocessing steps were difficult
* I knew from looking at other peoples notebooks in kaggle for this anime dataset that they had also
divided the problem into a subproblem such as only using anime movies
* the dataprocessing step where you had to cut-down the size of the ratings_df based on j=5 reviews randomly per unique movie, was difficult step and I got stuck there for a good while, until I searched on stack overflow
* overall, I believe that this notebook does exactly answer the questions 1,2 and 3 from the assignment requirements

### The main points on how to carry out this experiment without crashing the computer, were 
1. reduce the problem scope, focus into subproblem
2. reduce size of ratings_df if you must use this dataframe, it is must step
3. joining the dataframes was not particualrly useful except that it helped to make the "getting movie recommendation names" part little bit more straightforward because you simply execute "sql-like search" into the merged dataframe
such that you can retreive for each anime_id, the corresponding row, and from that row you select the corresponding movie name cell value

### other main problem was to decide if you wanted to have user-based KNN or item-based KNN 

* finding useful information that was understandable about this algorithm was difficult
* I looked for a "for dummies version" of KNN but couldnt find any good sources
* from what I saw about the assumptions of the user-based filtering is sounded a little bit BS to my un-educated first glance

Here is a quote from a seemingly authoritative source about user-based KNN for recommendation engines
I still didn't completely understand how the recommender is able to recommend any movies... and especially
for myself I would have  wanted to make a recommender engine, which is able to recommend individualized recommendations for new anime fans, who are anime noobs, who really need that advice on what is some good anime that they could watch based on movie genre that they typically like otherwise etc...

### in fact, I remained slightly hostile to the idea of the user-based recommendations, I would have liked to do the item-based one, but anyways... I did the user based

#### User-Based Collaborative Filtering (UB-CF)

> Imagine that we want to recommend a movie to our friend Stanley. We could assume that similar people will have similar taste. Suppose that me and Stanley have seen the same movies, and we rated them all almost identically. But Stanley hasn’t seen ‘The Godfather: Part II’ and I did. If I love that movie, it sounds logical to think that he will too. With that, we have created an artificial rating based on our similarity.

> Well, UB-CF uses that logic and recommends items by finding similar users to the active user (to whom we are trying to recommend a movie). A specific application of this is the user-based Nearest Neighbor algorithm. This algorithm needs two tasks: